## NARR STP Annual Cycles
##### Dr. Victor Gensini (Fall 2017)
##### Department of Geographic and Atmospheric Sciences
<img style="float:left;height:40px"src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQthtdrc2lZyEOC1iiwubFyvNfzJE7q7_GCTdlFF3peKckmi7cN">

In [1]:
import datetime, os, calendar
import numpy as np
import pandas as pd
from netCDF4 import date2num,date2index
from netCDF4 import Dataset
import pickle

In [2]:
stp_yr = np.empty((0,277,349))
stp_hrs = np.empty((0,38,277,349))
stp_doy = np.empty((0,1,38,277,349))
nc = Dataset('stp_cin_1979_2016.nc','r',format='NETCDF4_CLASSIC')

In [3]:
###############################
#Enter the start and end date:#
######################################
#start_input = '1979110100' #YYYYMMDDHH
#end_input  =  '2016110100' #YYYYMMDDHH
######################################
hours=['00','03','06','09','12','15','18','21']
hours=['21']
months=['01','02','03','04','05','06','07','08','09','10','11','12']
for month in months:
    if month=='01' or month=='03' or month=='05' or month=='07' or month=='08' or month=='10' or month=='12':
        emon = 31
    elif month=='04' or month=='06' or month=='09' or month=='11':
        emon = 30
    if month=='02':
        emon = 28
    daycount=1
    while daycount <= emon:
        for hrs in hours:
            start_input = '1979'+month+'%02d'%daycount+hrs    #YYYYMMDDHH
            end_input  =  '2016'+month+'%02d'%daycount+hrs #YYYYMMDDHH
            begdate = datetime.datetime.strptime(start_input,"%Y%m%d%H")
            enddate = datetime.datetime.strptime(end_input,"%Y%m%d%H")
            dates = []
            while begdate <= enddate:
                if calendar.isleap(begdate.year) == True:   
                    dates.append(begdate)
                    begdate+=datetime.timedelta(days=1)
                else:
                    dates.append(begdate)
                begdate+=datetime.timedelta(days=365)
            #do things here
            #print dates
            #print len(dates)
            for i,dt in enumerate(dates):
                idex = date2index(dt,nc['time'])
                stp = nc.variables["stp"][idex][:][:]
                cin  = nc.variables["sbcin"][idex][:][:]
                #cin mask
                term5 = np.fabs(cin)
                term5[np.fabs(cin)>50]=0.
                term5[np.fabs(cin)<=50]=1.
                stp = stp * term5
                stp[stp<1]=0.
                stp[stp>=1]=1.
                stp_yr = np.append(stp_yr,[stp],axis=0)
                #print scp_climo.shape
            #print stp_yr.shape
            #print stp_hrs.shape
            del idex
            del stp
            del cin
            del term5
            stp_hrs=np.append(stp_hrs,[stp_yr],axis=0)
            del stp_yr
            stp_yr = np.empty((0,277,349))
        #print stp_hrs.shape
        #print stp_doy.shape
        stp_doy=np.append(stp_doy,[stp_hrs],axis=0)
        del stp_hrs
        stp_hrs = np.empty((0,38,277,349))
        daycount+=1
print (stp_doy.shape)
stp_doy = np.sum(stp_doy,axis=2)
np.save('stp_21z_narr.npy',stp_doy)

(365, 1, 38, 277, 349)


In [4]:
#nc.close()
#mean_stp_daily = np.mean(scp_final,axis=1)
#stdev_scp_daily = np.std(scp_final,axis=1)
#log_values = np.log(scp_final)
#mean_scp_daily_log = np.mean(log_values,axis=1)
#stdev_scp_daily_log = np.std(log_values,axis=1)
#print mean_scp_daily.shape
#print stdev_scp_daily.shape
#stp_doy.dump('narr_00z_stp.npy') #Daily Mean
#stdev_scp_daily.dump('narr_stdev_scp_daily.npy') #Daily STDEV
#mean_scp_daily_log.dump('narr_mean_scp_daily_log.npy') 
#stdev_scp_daily_log.dump('narr_stdev_scp_daily_log.npy') 